In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import time
import importlib

In [2]:
import sys
sys.path.append("../../../libs_bell")
from lib_preprocessing import job_ids_to_result, flatten_results_jobs_list, arrange_results_list_tensored3, results_list_to_hist_list

In [3]:
with open("pkls/raw_hist_list.pkl", "rb") as f:
    raw_hist_list = pickle.load(f)
with open("pkls/results_meas_cal.pkl", "rb") as f:
    results_meas_cal = pickle.load(f)

In [4]:
from libs_qrem import DeltaFilter
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter

In [5]:
max_size = 65
max_length = 64
delta_mitigator_list = []
for n in range(2, max_size + 1):
    mit_pattern = [[i] for i in range(n)]
    meas_fitter = TensoredMeasFitter(results_meas_cal[n - 1], mit_pattern=mit_pattern)
    delta_mitigator_list.append(DeltaFilter(n, meas_fitter.cal_matrices))
    if n % 10 == 0:
        print("size", n, "finished")
print("length of delta_mitigator_list: ", len(delta_mitigator_list))

size 10 finished
size 20 finished
size 30 finished
size 40 finished
size 50 finished
size 60 finished
length of delta_mitigator_list:  64


In [6]:
for i in range(max_length):
    t1 = time.perf_counter()
    _ = delta_mitigator_list[i].apply(raw_hist_list[i + 1])
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits,", t2 - t1, "s )")

1 th finished ( 2 qubits, 7.171099999858654e-05 s )
2 th finished ( 3 qubits, 6.227899999977637e-05 s )
3 th finished ( 4 qubits, 0.00011096199999904854 s )
4 th finished ( 5 qubits, 0.00020284700000061662 s )
5 th finished ( 6 qubits, 0.0002829729999991315 s )
6 th finished ( 7 qubits, 0.0004664329999997108 s )
7 th finished ( 8 qubits, 0.0008479230000002502 s )
8 th finished ( 9 qubits, 0.0009349419999988839 s )
9 th finished ( 10 qubits, 0.0014427520000008798 s )
10 th finished ( 11 qubits, 0.001774237000001122 s )
11 th finished ( 12 qubits, 0.0024321720000006763 s )
12 th finished ( 13 qubits, 0.005103704999999792 s )
13 th finished ( 14 qubits, 0.006526345000001044 s )
14 th finished ( 15 qubits, 0.010133948000000004 s )
15 th finished ( 16 qubits, 0.011417444000000998 s )
16 th finished ( 17 qubits, 0.01953271400000034 s )
17 th finished ( 18 qubits, 0.023622174999999856 s )
18 th finished ( 19 qubits, 0.03208419799999973 s )
19 th finished ( 20 qubits, 0.04017053099999934 s )
2

In [7]:
delta_mitigator_info = []
for i in range(max_length):
    t1 = time.perf_counter()
    delta_mitigator_info.append({"exact_one_norm_of_reduced_inv_A": delta_mitigator_list[i].exact_one_norm_of_reduced_inv_A(),
                                 "mitigated_hist": delta_mitigator_list[i].mitigated_hist(),
                                 "x_s": delta_mitigator_list[i].x_s(),
                                 "x_hat": delta_mitigator_list[i].x_hat(),
                                 "x_tilde": delta_mitigator_list[i].x_tilde(),
                                 "sum_of_x": delta_mitigator_list[i].sum_of_x(),
                                 "sum_of_x_hat": delta_mitigator_list[i].sum_of_x_hat(),
                                 "sum_of_x_tilde": delta_mitigator_list[i].sum_of_x_tilde(),
                                 "indices_to_keys_vector": delta_mitigator_list[i].indices_to_keys_vector(),
                                 "times": delta_mitigator_list[i].times(),
                                 "expval": delta_mitigator_list[i].expval(),
                                 "mitigation_stddev": delta_mitigator_list[i].mitigation_stddev(norm_type = "exact"),
                                 })
    t2 = time.perf_counter()
    print(i + 1, "th finished (", i + 2, "qubits,", t2 - t1, "s )")

1 th finished ( 2 qubits, 0.00027375400000551053 s )
2 th finished ( 3 qubits, 0.0001514140000011821 s )
3 th finished ( 4 qubits, 8.429999999748361e-05 s )
4 th finished ( 5 qubits, 0.00017816300000106366 s )
5 th finished ( 6 qubits, 0.00011572900000089703 s )
6 th finished ( 7 qubits, 0.00024142899999901601 s )
7 th finished ( 8 qubits, 0.0018336060000052612 s )
8 th finished ( 9 qubits, 0.0017309209999893937 s )
9 th finished ( 10 qubits, 0.00030384800000149426 s )
10 th finished ( 11 qubits, 0.00020035600000767317 s )
11 th finished ( 12 qubits, 0.0003631910000052585 s )
12 th finished ( 13 qubits, 0.0005376439999906779 s )
13 th finished ( 14 qubits, 0.00045288500000140175 s )
14 th finished ( 15 qubits, 0.0005624190000048657 s )
15 th finished ( 16 qubits, 0.0006524319999954287 s )
16 th finished ( 17 qubits, 0.0008927389999939805 s )
17 th finished ( 18 qubits, 0.003266511999996169 s )
18 th finished ( 19 qubits, 0.0008867850000058297 s )
19 th finished ( 20 qubits, 0.001198986

In [8]:
with open("./pkls/delta_mitigator_info.pkl", "wb") as f:
    pickle.dump(delta_mitigator_info, f)